In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class StrongKeyMixing(nn.Module):
    """Stronger key-dependent transformation"""

    def __init__(self, dim):
        super(StrongKeyMixing, self).__init__()
        self.key_proj1 = nn.Linear(1, dim)
        self.key_proj2 = nn.Linear(1, dim)
        self.mixing = nn.Linear(dim * 2, dim)
        self.gate = nn.Linear(dim, dim)

    def forward(self, x, key):
        """x: (B, L, D), key: (B, L)"""
        k_exp = key.unsqueeze(-1)

        # Multiple key projections
        k1 = torch.tanh(self.key_proj1(k_exp))
        k2 = torch.sigmoid(self.key_proj2(k_exp))

        # Gated mixing
        gate = torch.sigmoid(self.gate(x))
        x_keyed = x * k1 + (1 - gate) * k2

        # Non-linear mixing
        combined = torch.cat([x, x_keyed], dim=-1)
        mixed = self.mixing(combined)

        return torch.tanh(mixed)


class MultiLayerKeyEncryption(nn.Module):
    """Multiple encryption layers with different keys"""

    def __init__(self, dim, num_key_layers=4):
        super(MultiLayerKeyEncryption, self).__init__()
        self.key_layers = nn.ModuleList([
            StrongKeyMixing(dim) for _ in range(num_key_layers)
        ])
        self.norms = nn.ModuleList([
            nn.LayerNorm(dim) for _ in range(num_key_layers)
        ])

    def forward(self, x, keys):
        """Apply multiple key-dependent transformations"""
        for i, (key_layer, norm) in enumerate(zip(self.key_layers, self.norms)):
            key_idx = min(i, len(keys) - 1)
            x = key_layer(x, keys[key_idx])
            x = norm(x)
        return x


class ImprovedAliceEncoder(nn.Module):
    """Stronger encryption with multiple key-dependent layers"""

    def __init__(self, vocab_size=98, embed_dim=128, num_layers=4, max_len=128):
        super(ImprovedAliceEncoder, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        # Initial projection
        self.input_proj = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.LayerNorm(embed_dim * 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(embed_dim * 2, embed_dim),
            nn.LayerNorm(embed_dim)
        )

        # Strong key-dependent encryption
        self.key_encryption = MultiLayerKeyEncryption(embed_dim, num_key_layers=4)

        # Additional encoding layers
        self.encoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=4,
                dim_feedforward=embed_dim * 4,
                dropout=0.1,
                activation='gelu',
                batch_first=True
            ) for _ in range(num_layers)
        ])

        # Final non-linear transformation
        self.output_proj = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.Tanh()
        )

    def forward(self, tokens, keys):
        """
        tokens: (B, L)
        keys: list of (B, L) tensors
        """
        # Embed and project
        x = self.embedding(tokens)
        x = self.input_proj(x)

        # Apply key-dependent encryption (critical!)
        x = self.key_encryption(x, keys)

        # Transformer encoding
        for layer in self.encoder_layers:
            x = layer(x)

        # Final projection
        x = self.output_proj(x)

        return x


class AliceEncryptor:
    """Wrapper for Alice encoder"""

    def __init__(self, vocab_size=98, embed_dim=128, num_layers=4, max_len=128,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.max_len = max_len
        self.alice = ImprovedAliceEncoder(vocab_size, embed_dim, num_layers, max_len).to(device)

        from v3_part1 import StringProcessor
        self.processor = StringProcessor(max_length=max_len)

    def encrypt(self, messages, keys):
        """Encrypt messages"""
        if isinstance(messages, list) and isinstance(messages[0], str):
            tokens = self.processor.batch_encode(messages).to(self.device)
        else:
            tokens = messages.to(self.device)

        seq_len = tokens.size(1)
        truncated_keys = [k[:, :seq_len] for k in keys['key_tensors']]

        self.alice.eval()
        with torch.no_grad():
            encrypted = self.alice(tokens, truncated_keys)

        return encrypted

In [5]:
if __name__ == "__main__":
    print("="*70)
    print("IMPROVED ALICE ENCODER V3")
    print("="*70)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    from v3_part1 import StringProcessor, KeyManager

    processor = StringProcessor(max_length=64)
    key_manager = KeyManager(device=device)
    alice_enc = AliceEncryptor(
        vocab_size=processor.vocab_size,
        embed_dim=128,
        num_layers=4,
        max_len=64,
        device=device
    )

    messages = [
        "Hello World!",
        "Testing encryption system.",
        "Short test"
    ]

    print(f"\nTest messages:")
    for i, msg in enumerate(messages):
        print(f"  {i+1}. '{msg}'")

    keys = key_manager.generate_keys_for_batch(
        batch_size=len(messages),
        seq_length=64
    )

    print("\nEncrypting...")
    encrypted = alice_enc.encrypt(messages, keys)

    print(f"\nEncryption complete!")
    print(f"  Output shape: {encrypted.shape}")
    print(f"  Sample encrypted: {encrypted[0, 0, :10]}")

    print("\n" + "="*70)

IMPROVED ALICE ENCODER V3

Test messages:
  1. 'Hello World!'
  2. 'Testing encryption system.'
  3. 'Short test'

Encrypting...

Encryption complete!
  Output shape: torch.Size([3, 64, 128])
  Sample encrypted: tensor([-0.7602,  0.7383, -0.4264, -0.2587,  0.1478,  0.2741, -0.5531, -0.5341,
         0.4954,  0.1915])

